In [1]:
import pandas as pd
store = pd.HDFStore('Seeds.h5')
Seeds = store["Seeds_7_25_16"]

# Peer Review
### Prior to sending data to RAC we require an independent review

Reviewer:
  - Spot check ARERR tables used in validation.   
    - [ARERR Tables](#ARERR-Comparisons)  
  - Ensure comparison tables at bottom of document show good data match  
    - [2014](#2014-Validation)  
    - [2015](#2015-Validation)  
  - Review logic for calculating releases from release rates and release dates  
    - [Logic](#Calculate-Releases)  
  - Review schema transformation and ensure new schema matches RAC requirements  
    - [Transformation](#Schema-Transformation)  
  - Review the Final Seeds File "OysterCreek_SEEDS_ver_2.00"  
    - [Final File](#Final-Seeds-File)  


### Wrangling OC SEEDS Data ###
The OC SEEDS data was delivered in [612 plain text files.](https://chemstaff.egnyte.com/fl/WOr1mKyo0f)

### The text files were parsed with the following code. ###
Commented out after initial parsing was completed.

In [2]:
# filenames = !dir /b C:\OC  

# SeedsData = {}
# for file in filenames:
#     seeds = open('C:\\OC\\' + file)
#     text = seeds.read().replace("\n"," ").replace(","," ").split()
#     if len(text[3]) == 1:
#         text.insert(1,None)
#     while len(text[1:]) < 21:
#         text.append(None)
#     SeedsData[text[0]] = text[1:]

In [3]:
print("SEEDS data starts on",Seeds.STARTDATE.min(),"and ends on",Seeds.STOPDATE.max())

SEEDS data starts on 2014-01-01 00:00:00 and ends on 2016-06-07 09:28:00


Convert Nuclide names to standard format

In [4]:
old = ['CO60','I131', 'C14', 'MO99', 'H3', 'MN54', 'KR88', 'KR85M',
       'CR51', 'CO58', 'ZN65', 'SR89', 'CS137', 'XE133', 'BA140', 'I133',
       'XE138', 'KR87', 'XE135', 'I135']
new = ["Co-60",'I-131', 'C-14', 'Mo-99', 'H-3', 'Mn-54', 'Kr-88', 'Kr-85m',
       'Cr-51', 'Co-58', 'Zn-65', 'Sr-89', 'Cs-137', 'Xe-133', 'Ba-140', 'I-133',
       'Xe-138', 'Kr-87', 'Xe-135', 'I-135']
for name in range(len(old)):
    Seeds.loc[Seeds["NUCLIDE"] == old[name],"NUCLIDE"] = new[name]

# ARERR Comparisons
### Import 2014 and 2015 ARERR
All data is for gaseous continuous releases.  There are no batch releases.

In [5]:
data = {(2015,"Elevated","Kr-85m") : [2.1E-1, 4.14E-1, 9.07E-1, 1.19],
        (2015,"Elevated","Kr-88") : [1.38E-1, 3.17E-1, 7.07E-1, 1.02],
        (2015,"Elevated","Xe-133") : [3.12E-1, 4.5E-1, 1.03, 1.25],
        (2015,"Elevated","Xe-135") : [13.7, 13.1, 3.88, 1.62],
       (2015,"Elevated","I-131") : [1.99E-4, 3.91E-4, 3.32E-4, 2.3E-4],
       (2015,"Elevated","I-133") : [3.38E-4, 1.1E-3, 9.86E-4, 7.17E-4],
       (2015,"Elevated","Sr-89") : [1.0E-3, 3.18E-4, 1.09E-4, 1.42E-4],
       (2015,"Elevated","Cs-137") : [6.64E-4,None,None,None],
       (2015,"Elevated","Ba-140") : [1.27E-3, 5.25E-4, 1.03E-4, 1.37E-4],
       (2015,"Elevated","Cr-51") : [3.29E-4, 1.01E-4,None,None],
       (2015,"Elevated","Mn-54") : [5.68E-4,4.83E-4,2.63E-4,2.05E-4],
       (2015,"Elevated","Co-58") : [4.93E-3,1.63E-3,1.31E-3,9.71E-4],
       (2015,"Elevated","Co-60") : [4.26E-03, 1.93E-03, 1.71E-03, 1.41E-03],
       (2015,"Elevated","Mo-99") : [1.92E-04, 1.00E-04, 5.01E-05, 7.05E-06 ],
       (2015,"Elevated","Zn-65") : [1.04E-03, 5.10E-04, 2.59E-04, 2.40E-04 ],
       (2015,"Elevated","H-3") : [6.45, 1.07E+01, 6.48, 4.96],
       (2015,"Elevated","C-14") : [2.18, 2.14, 2.37, 2.35],
        (2015,"Ground","Kr-85m") : [None, 2.75E-02, None, None],
        (2015,"Ground","Kr-87") : [None, 3.68E-01, None, None],
        (2015,"Ground","Kr-88") : [None, 6.56E-02, None, None],
        (2015,"Ground","Xe-135") : [None, 5.40E-01, None, None],
        (2015,"Ground","I-131") : [None, 7.17E-07, 1.17E-06, 3.39E-07], 
        (2015,"Ground","I-133") : [None, 1.69E-06, 1.52E-06, 1.69E-08],
        (2015,"Ground","Mn-54") : [None, 1.09E-06, None, None],
        (2015,"Ground","Co-58") : [6.18E-07, 1.07E-05, 1.91E-05, 3.00E-06], 
        (2015,"Ground","Co-60") : [4.91E-06, 2.29E-05, 3.14E-05, 5.31E-06],
        (2015,"Ground","H-3") : [6.20E-01, 5.95E-01, 4.82E-01, 4.93E-01],
        (2015,"Ground","C-14") : [6.76E-02, 6.63E-02, 7.32E-02, 7.28E-02],
        (2014,"Elevated","Kr-85m") : [1.90, 1.51, 8.55E-01, 4.38E-01],
        (2014,"Elevated","Kr-87") :  [3.24, 1.08, 2.39, None],
        (2014,"Elevated","Kr-88") :  [2.13, 2.26, 5.80E-01, 1.98],
        (2014,"Elevated","Xe-133") :  [1.45, 1.70, 1.06, 5.25E-01],
        (2014,"Elevated","Xe-135") :  [5.96, 1.11E+01, 8.71, 4.82],
        (2014,"Elevated","Xe-138") :  [4.83, 1.71, None, None],
        (2014,"Elevated","I-131") :  [3.88E-04, 6.18E-04, 6.19E-04, 7.03E-05],
        (2014,"Elevated","I-133") :  [8.14E-04, 1.76E-03, 9.50E-04, 5.20E-05],
        (2014,"Elevated","Sr-89") :  [6.68E-04, 1.57E-04, 2.93E-04, 6.45E-05],
        (2014,"Elevated","Cs-137") :  [1.92E-05, None, None, None],
        (2014,"Elevated","Ba-140") :  [1.13E-03, 4.17E-04, 7.12E-04, 3.70E-04],
        (2014,"Elevated","Cr-51") :  [None, None, 7.77E-05, 2.09E-05],
        (2014,"Elevated","Mn-54") :  [5.40E-04, 4.16E-04, 1.14E-03, 3.98E-04],
        (2014,"Elevated","Co-58") :  [2.41E-03, 1.97E-03, 3.96E-03, 2.0E-03],
        (2014,"Elevated","Co-60") :  [3.04E-03, 2.30E-03, 5.01E-03, 1.88E-03],
        (2014,"Elevated","Mo-99") :  [2.46E-04, 2.25E-04, 3.52E-04, 5.20E-05],
        (2014,"Elevated","Zn-65") :  [8.40E-04, 7.41E-04, 1.38E-03, 2.36E-04 ],
        (2014,"Elevated","H-3") :  [6.17, 9.80, 7.49, 4.40],
        (2014,"Elevated","C-14") :  [2.32, 2.33, 1.76, 1.98],
        (2014,"Ground","Xe-135") : [None, 1.34, 9.26E-01, None],
        (2014,"Ground","I-131") : [None, 9.91E-07, 1.30E-05, 1.52E-06],
        (2014,"Ground","I-133") : [None, 4.86E-06, 1.94E-06, None],
        (2014,"Ground","Cs-137") : [8.29E-07, 8.38E-07, 2.27E-07, None],
        (2014,"Ground","Ba-140") : [1.19E-05, 1.20E-05, 3.26E-06, None],
        (2014,"Ground","Cr-51") : [6.72E-06, 6.79E-06, 1.84E-06, None],
        (2014,"Ground","Mn-54") : [8.03E-06, 8.12E-06, 2.20E-06, None],
        (2014,"Ground","Co-58") : [2.30E-05, 2.36E-05, 6.65E-06, 7.76E-07],
        (2014,"Ground","Co-60") : [2.81E-05, 3.78E-05, 1.94E-05, 2.18E-06],
        (2014,"Ground","Mo-99") : [2.21E-06, 2.24E-06, 6.06E-07, None],
        (2014,"Ground","Zn-65") : [8.71E-06, 8.81E-06, 2.38E-06, None],
        (2014,"Ground","H-3") : [2.66E-01, 2.03E-01, 4.40E-01, 1.70E-01],
        (2014,"Ground","C-14") : [7.20E-02, 7.21E-02, 5.44E-02, 6.13E-02]}

In [6]:
ARERR = pd.DataFrame(data, index=["Q1", "Q2", "Q3", "Q4"]).T

In [7]:
ARERR["Q1"] = pd.to_numeric(ARERR["Q1"])
ARERR["Q2"] = pd.to_numeric(ARERR["Q2"])
ARERR["Q3"] = pd.to_numeric(ARERR["Q3"])
ARERR["Q4"] = pd.to_numeric(ARERR["Q4"])

In [8]:
ARERR["TOTAL"] = ARERR["Q1"].fillna(value=0) + ARERR["Q2"].fillna(value=0) +\
ARERR["Q3"].fillna(value=0) + ARERR["Q4"].fillna(value=0)

# Calculate Releases
Curies Released = Duration of Release in seconds * Release Rate (Microcuries / Sec)

In [9]:
def seconds(value):
    return value.total_seconds()

In [10]:
Seeds["RELEASED"] = Seeds["MICROCURIES"] * (Seeds["STOPDATE"] - Seeds["STARTDATE"]).apply(seconds) / 1000000
# Seeds.loc[Seeds["REL_MODE_OR_DIL"] == 1,"REL_MODE_OR_DIL"] = "Elevated"
# Seeds.loc[Seeds["REL_MODE_OR_DIL"] == 2,"REL_MODE_OR_DIL"] = "Ground"

# 2014 Validation

In [11]:
Atime0 = Seeds["STARTDATE"] > pd.datetime(2013, 12, 31, 23, 59)
Atime1 = Seeds["STARTDATE"] < pd.datetime(2015, 1, 1, 0, 0)
df14 = Seeds[Atime0 & Atime1]
SEEDS_Tot_Nuclide_2014 = df14.groupby(by=["REL_MODE_OR_DIL","NUCLIDE"]).sum()["RELEASED"]
print("2014 SEEDS data starts on",df14.STARTDATE.min(),"and ends on",df14.STOPDATE.max())
dfARERR14 =  ARERR.loc[2014,slice(None),slice(None)]["TOTAL"].reset_index()
dfARERR14.columns = ["Year","REL_MODE_OR_DIL","NUCLIDE","RELEASED"]
dfARERR14 = dfARERR14[["REL_MODE_OR_DIL","NUCLIDE","RELEASED"]]
pd.merge(dfARERR14,SEEDS_Tot_Nuclide_2014.reset_index(), right_on=["REL_MODE_OR_DIL","NUCLIDE"] ,
         left_on=["REL_MODE_OR_DIL","NUCLIDE"], suffixes=('_ARERR','_SEEDS'),
         how='outer').sort_values(by=["REL_MODE_OR_DIL","NUCLIDE"]).fillna("").reset_index(drop=True)

2014 SEEDS data starts on 2014-01-01 00:00:00 and ends on 2014-12-31 23:59:00


,REL_MODE_OR_DIL,NUCLIDE,RELEASED_ARERR,RELEASED_SEEDS
0,Elevated,Ba-140,0.002629,2.629145e-03
1,Elevated,C-14,8.39,8.400941e+00
2,Elevated,Co-58,0.01034,1.034158e-02
3,Elevated,Co-60,0.01223,5.246105e+06
4,Elevated,Cr-51,9.86e-05,9.859662e-05
5,Elevated,Cs-137,1.92e-05,1.920021e-05
6,Elevated,H-3,27.86,2.786000e+01
7,Elevated,I-131,0.0016953,1.695158e-03
8,Elevated,I-133,0.003576,3.578646e-03
9,Elevated,Kr-85m,4.703,4.703155e+00


# 2015 Validation

In [12]:
Btime0 = Seeds["STARTDATE"] > pd.datetime(2014, 12, 31, 23, 59)
Btime1 = Seeds["STARTDATE"] < pd.datetime(2016, 1, 1, 0, 0)
df15 = Seeds[Btime0 & Btime1]
SEEDS_Tot_Nuclide_2015 = df15.groupby(by=["REL_MODE_OR_DIL","NUCLIDE"]).sum()["RELEASED"]
print("2015 SEEDS data starts on",df15.STARTDATE.min(),"and ends on",df15.STOPDATE.max())
dfARERR15 = ARERR.loc[2015,slice(None),slice(None)]["TOTAL"].reset_index()
dfARERR15.columns = ["Year","REL_MODE_OR_DIL","NUCLIDE","RELEASED"]
dfARERR15 = dfARERR15[["REL_MODE_OR_DIL","NUCLIDE","RELEASED"]]
pd.merge(dfARERR15,SEEDS_Tot_Nuclide_2015.reset_index(), right_on=["REL_MODE_OR_DIL","NUCLIDE"] ,
         left_on=["REL_MODE_OR_DIL","NUCLIDE"], suffixes=('_ARERR','_SEEDS'),
         how='outer').sort_values(by=["REL_MODE_OR_DIL","NUCLIDE"]).fillna("").reset_index(drop=True)

2015 SEEDS data starts on 2015-01-01 00:00:00 and ends on 2015-12-31 23:29:00


,REL_MODE_OR_DIL,NUCLIDE,RELEASED_ARERR,RELEASED_SEEDS
0,Elevated,Ba-140,0.002035,0.002035
1,Elevated,C-14,9.04,9.046000
2,Elevated,Co-58,0.008841,0.008839
3,Elevated,Co-60,0.00931,0.009312
4,Elevated,Cr-51,0.00043,0.000431
5,Elevated,Cs-137,0.000664,0.000664
6,Elevated,H-3,28.59,28.560002
7,Elevated,I-131,0.001152,0.001152
8,Elevated,I-133,0.003141,0.003141
9,Elevated,Kr-85m,2.721,2.700782


# Schema Transformation


In [14]:
Seeds.head()

,PT_NUMBER,FLOW,STARTDATE,STOPDATE,REL_MODE_OR_DIL,NUCLIDECOUNT,NUCLIDE,MICROCURIES,RELEASED
0,EFLEAKS4,143.0,2014-01-01,2014-07-25 15:00:00,Ground,8,Co-60,3.428313e-06,0.000061
1,STK0114C,176000.0,2014-01-01,2014-01-03 07:00:00,Elevated,1,I-131,3.214432e-05,0.000006
2,L0114COM,40300.0,2014-01-01,2014-01-31 23:59:00,Ground,1,C-14,9.257836e-03,0.024796
3,EFLEAKS4,143.0,2014-01-01,2014-07-25 15:00:00,Ground,8,Mo-99,2.847929e-07,0.000005
4,T14H3JAN,200000.0,2014-01-01,2014-01-31 23:59:00,Ground,1,H-3,3.199743e-02,0.085700


### RAC Prefered Schema
|PT_NUMBER|PLANT|SRC_UNIT|NUCLIDE|ACTIVITY|ACT_UNITS|Max0Avg1Sum2|
|-|-|-|-|-|-|-|
|NUCTYPEIMPTYPE|SOURCE|STARTDATE|ENDDATE|DURATION_MIN|FLOW_RATE|FLOW_UNITS|
|REL_VOL|REL_UNITS|RELEASE_SOURCE|DISCHARGE_POINT|REL_MODE_OR_DIL|IS_MDA|
|BATCHORCONT|ALLOCATION_PERCENT|

In [16]:
def Minutes(value):
    return value.total_seconds() / 60

In [23]:
Seeds["PLANT"] = "Oyster Creek"
Seeds["SRC_UNIT"] = 1
Seeds["ACTIVITY"] = Seeds["MICROCURIES"]
Seeds["ACT_UNITS"] = "Microcuries per Second"
Seeds["Max0Avg1Sum2"] = None
Seeds["NUCTYPEIMPTYPE"] = Seeds["NUCLIDECOUNT"]
Seeds["SOURCE"] = None
Seeds["DURATION_MIN"] = (Seeds["STOPDATE"] - Seeds["STARTDATE"]).apply(Minutes)
Seeds["FLOW_RATE"] = Seeds["FLOW"]
Seeds["FLOW_UNITS"] = None
Seeds["REL_VOL"] = None
Seeds["REL_UNITS"] = None
Seeds["RELEASE_SOURCE"] = None
Seeds["DISCHARGE_POINT"] = None
Seeds["IS_MDA"] = None
Seeds["BATCHORCONT"] = "C"
Seeds["ALLOCATION_PERCENT"] = 100
Seeds["ENDDATE"] = Seeds["STOPDATE"]

In [24]:
Seeds = Seeds[["PT_NUMBER","PLANT","SRC_UNIT","NUCLIDE","ACTIVITY","ACT_UNITS","Max0Avg1Sum2","NUCTYPEIMPTYPE",
              "SOURCE","STARTDATE","ENDDATE","DURATION_MIN","FLOW_RATE","FLOW_UNITS","REL_VOL","REL_UNITS",
              "RELEASE_SOURCE","DISCHARGE_POINT","REL_MODE_OR_DIL","IS_MDA","BATCHORCONT","ALLOCATION_PERCENT"]]

In [49]:
Seeds.loc[Seeds["PT_NUMBER"].str.startswith("A"),"RELEASE_SOURCE"] = "Augmented Off Gas"
Seeds.loc[Seeds["PT_NUMBER"].str.contains("LEAKS"),"RELEASE_SOURCE"] = "Abnormal Release"
Seeds.loc[Seeds["PT_NUMBER"].str.startswith("L"),"RELEASE_SOURCE"] = "Turbine Bldg Lube Oil Bay"
Seeds.loc[Seeds["PT_NUMBER"].str.startswith("S"),"RELEASE_SOURCE"] = "Stack"
Seeds.loc[Seeds["PT_NUMBER"].str.startswith("T"),"RELEASE_SOURCE"] = "Turbine Bldg"
Seeds.loc[Seeds["PT_NUMBER"].str.startswith("TBNG"),"RELEASE_SOURCE"] = "Turbine Bldg Noble Gas"

In [50]:
Seeds[Seeds["RELEASE_SOURCE"].isnull()]

,PT_NUMBER,PLANT,SRC_UNIT,NUCLIDE,ACTIVITY,ACT_UNITS,Max0Avg1Sum2,NUCTYPEIMPTYPE,SOURCE,STARTDATE,...,FLOW_RATE,FLOW_UNITS,REL_VOL,REL_UNITS,RELEASE_SOURCE,DISCHARGE_POINT,REL_MODE_OR_DIL,IS_MDA,BATCHORCONT,ALLOCATION_PERCENT


In [51]:
Seeds.columns

Index(['PT_NUMBER', 'PLANT', 'SRC_UNIT', 'NUCLIDE', 'ACTIVITY', 'ACT_UNITS',
       'Max0Avg1Sum2', 'NUCTYPEIMPTYPE', 'SOURCE', 'STARTDATE', 'ENDDATE',
       'DURATION_MIN', 'FLOW_RATE', 'FLOW_UNITS', 'REL_VOL', 'REL_UNITS',
       'RELEASE_SOURCE', 'DISCHARGE_POINT', 'REL_MODE_OR_DIL', 'IS_MDA',
       'BATCHORCONT', 'ALLOCATION_PERCENT'],
      dtype='object')

In [52]:
Seeds.loc[0]

PT_NUMBER                           EFLEAKS4
PLANT                           Oyster Creek
SRC_UNIT                                   1
NUCLIDE                                Co-60
ACTIVITY                         3.42831e-06
ACT_UNITS             Microcuries per Second
Max0Avg1Sum2                            None
NUCTYPEIMPTYPE                             8
SOURCE                                  None
STARTDATE                2014-01-01 00:00:00
ENDDATE                  2014-07-25 15:00:00
DURATION_MIN                          296100
FLOW_RATE                                143
FLOW_UNITS                              None
REL_VOL                                 None
REL_UNITS                               None
RELEASE_SOURCE              Abnormal Release
DISCHARGE_POINT                         None
REL_MODE_OR_DIL                       Ground
IS_MDA                                  None
BATCHORCONT                                C
ALLOCATION_PERCENT                       100
Name: 0, d

In [53]:
store['OC_SEEDS_ver_2.00'] = Seeds

C:\Users\collins\AppData\Local\Continuum\Anaconda3\lib\site-packages\tables\path.py:100: NaturalNameWarning: object name is not a valid Python identifier: 'OC_SEEDS_ver_2.00'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
C:\Users\collins\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['PT_NUMBER', 'PLANT', 'NUCLIDE', 'ACT_UNITS', 'Max0Avg1Sum2', 'SOURCE', 'FLOW_UNITS', 'REL_VOL', 'REL_UNITS', 'RELEASE_SOURCE', 'DISCHARGE_POINT', 'REL_MODE_OR_DIL', 'IS_MDA', 'BATCHORCONT']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [54]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: Seeds.h5
/OC_SEEDS_ver_2.00            frame        (shape->[1,22])  
/Seeds_7_21_16                frame        (shape->[1531,8])
/Seeds_7_24_16                frame        (shape->[1531,8])
/Seeds_7_25_16                frame        (shape->[1516,9])

# Final Seeds File
## [Final Seeds File](https://chemstaff.egnyte.com/dl/DrJWmKuoNR)

In [56]:
nuc = Seeds["NUCLIDE"] == "Kr-88"
ty = Seeds["REL_MODE_OR_DIL"] == "Ground"
Seeds[nuc & ty]

,PT_NUMBER,PLANT,SRC_UNIT,NUCLIDE,ACTIVITY,ACT_UNITS,Max0Avg1Sum2,NUCTYPEIMPTYPE,SOURCE,STARTDATE,...,FLOW_RATE,FLOW_UNITS,REL_VOL,REL_UNITS,RELEASE_SOURCE,DISCHARGE_POINT,REL_MODE_OR_DIL,IS_MDA,BATCHORCONT,ALLOCATION_PERCENT
567,TBNG0114,Oyster Creek,1,Kr-88,7.343283,Microcuries per Second,None,1,None,2014-11-12 02:35:00,...,40000.0,None,None,None,Turbine Bldg Noble Gas,None,Ground,None,C,100
925,AOGI0315,Oyster Creek,1,Kr-88,0.253664,Microcuries per Second,None,4,None,2015-05-25 04:10:00,...,8600.0,None,None,None,Augmented Off Gas,None,Ground,None,C,100
